In [1]:
%load_ext autoreload
%autoreload 2

import sklearn
import os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from scipy import interpolate
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

sys.path.insert(1, os.path.join(sys.path[0], ".."))
sys.path.append('../../')
from reproduction.pipeline.load import load_data
from reproduction import analysis

# Disable Warnings
import warnings; warnings.simplefilter('ignore')
tf.logging.set_verbosity(tf.logging.WARN)

In [2]:
def gridsearch(start, step, stop, max_samples=5000, sample_steps=4, trials=30):
    with open(ENCODER_DEF,"r") as f:
            encoder = tf.keras.models.model_from_json(f.read())
    encoder.load_weights(ENCODER_WEIGHTS)
    
    samples = np.logspace(np.log10(start+2), np.log10(max_samples), num=sample_steps).astype(int)
    search_results = [] # Force initialization
    # iterate on sampling 
    for i in samples:
        # iterate on clustering
        for j in range(start, stop, step):
                if i/j > 2: # Agglomerative clustering must have leaves with at least 2 elements
                    print('Samples: ', i,' Clusters: ',j)
                    minfoac = []
                    for trial in range(trials):
                        data = analysis.AEData(load_data(DATA, encoder.input_shape[1:]), n=i)
                        data.add_encoder(encoder)
                        N_CLUSTERS = j
                        ag1 = AgglomerativeClustering(n_clusters=N_CLUSTERS).fit_predict(data.encs[:int(i/2)])
                        ag2 = AgglomerativeClustering(n_clusters=N_CLUSTERS).fit_predict(data.encs)
                        minfoac.append(sklearn.metrics.adjusted_mutual_info_score(ag1, ag2[:int(i/2)]))
                    minfo_mean = np.nanmean(minfoac)
                    minfo_std = np.nanstd(minfoac)
                    search_results.append((i, N_CLUSTERS, minfo_mean, minfo_std))
                    print('Average Mutual information: ', minfo_mean, 'MI_STD: ', minfo_std, 'Precision: ', np.count_nonzero(~np.isnan(minfoac)))
    return search_results
    

In [3]:
DATA = "/project/foster/clouds/data/2015_05/*.tfrecord"
ENCODER_DEF = "/home/rlourenco/rdcep_clouds/output/m9-22_oceans/encoder.json"
ENCODER_WEIGHTS = "/home/rlourenco/rdcep_clouds/output/m9-22_oceans/encoder.h5"
# N_CLUSTERS = 10

In [ ]:
result = gridsearch(2,1,40,max_samples=10000, sample_steps=2)

Samples:  10000  Clusters:  2
Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Use `tf.data.experimental.shuffle_and_repeat(...)`.
Instructions for updating:
Use `tf.data.Dataset.batch(..., drop_remainder=True)`.
Average Mutual information:  0.5003018236603322 MI_STD:  0.1973245413495136 Precision:  30
Samples:  10000  Clusters:  3
Average Mutual information:  0.6117280764819709 MI_STD:  0.10466845268991093 Precision:  30
Samples:  10000  Clusters:  4
Average Mutual information:  0.6156232527212082 MI_STD:  0.07000680095354271 Precision:  30
Samples:  10000  Clusters:  5
Average Mutual information:  0.6416274012107064 MI_STD:  0.06972692468655274 Precision:  30
Samples:  10000  Clusters:  6
Average Mutual information:  0.6265519965508679 MI_STD:  0.041293782727401536 Precision:  30
Samples:  10000  Clusters:  7
Average Mutual information:  0.6520465048752868 MI_STD:  0.0384921270700231 Precision:  30
Samples:  10000  Clusters:  

In [ ]:
result = gridsearch(2,1,40,max_samples=10000, sample_steps=4)

Samples:  54  Clusters:  2
Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Use `tf.data.experimental.shuffle_and_repeat(...)`.
Instructions for updating:
Use `tf.data.Dataset.batch(..., drop_remainder=True)`.
Average Mutual information:  0.637374727264124 MI_STD:  0.32345079502786933 Precision:  30
Samples:  54  Clusters:  3
Average Mutual information:  0.7332971546702073 MI_STD:  0.20900686314266054 Precision:  30
Samples:  54  Clusters:  4
Average Mutual information:  0.7337189366528648 MI_STD:  0.1650970336124088 Precision:  30
Samples:  54  Clusters:  5
Average Mutual information:  0.7049696792139365 MI_STD:  0.11442450056664943 Precision:  30
Samples:  54  Clusters:  6
Average Mutual information:  0.7542339182793424 MI_STD:  0.13311748003695972 Precision:  30
Samples:  54  Clusters:  7
Average Mutual information:  0.7122616296128097 MI_STD:  0.12143520723315926 Precision:  30
Samples:  54  Clusters:  8
Average Mutual infor

Average Mutual information:  0.7477828977060859 MI_STD:  0.019549828799583228 Precision:  30
Samples:  736  Clusters:  39
Average Mutual information:  0.7592828468095654 MI_STD:  0.032759001947130655 Precision:  30
Samples:  10000  Clusters:  2
Average Mutual information:  0.5296795456765903 MI_STD:  0.1929682240383342 Precision:  30
Samples:  10000  Clusters:  3
Average Mutual information:  0.6048012516521581 MI_STD:  0.10163397375612432 Precision:  30
Samples:  10000  Clusters:  4
Average Mutual information:  0.6064302004689536 MI_STD:  0.08020232958258182 Precision:  30
Samples:  10000  Clusters:  5
Average Mutual information:  0.6178682870211566 MI_STD:  0.04727019334344826 Precision:  30
Samples:  10000  Clusters:  6
Average Mutual information:  0.6475343950799516 MI_STD:  0.045927190171492036 Precision:  30
Samples:  10000  Clusters:  7
Average Mutual information:  0.6462362170173777 MI_STD:  0.0455663834358556 Precision:  30
Samples:  10000  Clusters:  8
Average Mutual informati

In [ ]:
result
--

In [ ]:
df = pd.DataFrame(result)
df.columns = ["Samples", "Clusters", "MInfo"]

# Mutual Information Plots

### 28 Samples, 13 clusters maximum

In [ ]:
%matplotlib inline

plt.xlim(0,40)
plt.ylim(0,1)
y_28 =  df.loc[df['Samples'] == 28]
x_28 = y_28['Clusters']
plt.plot(x_28, y_28["MInfo"])
plt.show()


### 200 Samples, 40 clusters maximum

In [ ]:
%matplotlib inline

plt.xlim(0,40)
plt.ylim(0,1)
y_200 =  df.loc[df['Samples'] == 200]
x_200 = y_200['Clusters']
plt.plot(x_200, y_200["MInfo"])
plt.show()

### 1414 Samples, 40 clusters maximum

In [ ]:
%matplotlib inline

plt.xlim(0,40)
plt.ylim(0,1)
y_200 =  df.loc[df['Samples'] == 1414]
x_200 = y_200['Clusters']
plt.plot(x_200, y_200["MInfo"])
plt.show()

### 10000 Samples, 13 clusters maximum

In [ ]:
%matplotlib inline

plt.xlim(0,40)
plt.ylim(0,1)
y_200 =  df.loc[df['Samples'] == 10000]
x_200 = y_200['Clusters']
plt.plot(x_200, y_200["MInfo"])
plt.show()

#### Best result seems to be when using 200 samples, with 9 clusters, with a MI score of 0.84.

In [ ]:
---

In [ ]:
# result = gridsearch(2,1,500)
N_CLUSTERS = 9 #result[0]
# HiMI = #result[1]

In [ ]:
with open(ENCODER_DEF,"r") as f:
    encoder = tf.keras.models.model_from_json(f.read())
encoder.load_weights(ENCODER_WEIGHTS)

In [ ]:
data = analysis.AEData(load_data(DATA, encoder.input_shape[1:]), n=200)
data.add_encoder(encoder)

In [ ]:
m1 = AgglomerativeClustering(n_clusters=N_CLUSTERS)
m1.fit(data.encs[:100])
ag1 = AgglomerativeClustering(n_clusters=N_CLUSTERS).fit_predict(data.encs[:100])

m2 = AgglomerativeClustering(n_clusters=N_CLUSTERS)
m2.fit(data.encs)
ag2 = AgglomerativeClustering(n_clusters=N_CLUSTERS).fit_predict(data.encs)

In [ ]:
# Number of bins
nbins=N_CLUSTERS

plt.figure(figsize=(8,8))
plt.hist2d(ag1, ag2[:100], bins=nbins);
plt.title("Cluster overlap")
plt.xticks(range(0, nbins, 5)); plt.yticks(range(0, nbins, 5));
plt.xlabel("ag 10000"); plt.ylabel("ag 5000");
plt.colorbar()
print('Mutual Information score: ',sklearn.metrics.adjusted_mutual_info_score(ag1, ag2[:100]))

In [ ]:
overlap = data.imgs[:100,:,:,0][ag1 == ag2[:100]]
overlap_c = ag1[ag1 == ag2[:100]]



In [ ]:
fig, ax = plt.subplots(5, 8, figsize=(16, 10))

clu = data.imgs[:100,:,:,0][ag1 == 2]

for i, a in enumerate(ax.ravel()):
    a.imshow(clu[i], cmap="bone")
    a.set_xticks([])
    a.set_yticks([])



In [ ]:

from scipy.cluster.hierarchy import dendrogram

In [ ]:
def plot_dendrogram(model, **kwargs):

    # Children of hierarchical clustering
    children = model.children_

    # Distances between each pair of children
    # Since we don't have this information, we can use a uniform one for plotting
    distance = np.arange(children.shape[0])

    # The number of observations contained in each cluster level
    no_of_observations = np.arange(2, children.shape[0]+2)

    # Create linkage matrix and then plot the dendrogram
    linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)



That's hard to interpretate...

In [ ]:
plt.figure(figsize=(500,100))
plot_dendrogram(m2, labels=m2.labels_)